<a href="https://colab.research.google.com/github/ngonam2403/VEF-Final-Project_202007/blob/master/tweak_to_see.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def run_flow(train_X, train_y, algorithm_choice = None):
    X_train,X_test,y_train,y_test=train_test_split(
                                            train_X,
                                            train_y,
                                            test_size=0.25,
                                            random_state=0)
    # def train_model(train_X, train_Y, algorithm_choice):
    algorithm_choice.fit(X_train,y_train)
    predictions = algorithm_choice.predict(X_test)
    # def evaluate_model()
    df_confusion_matrix = metrics.confusion_matrix(y_test, predictions) 
    f1_score =  metrics.f1_score(predictions,y_test, average = 'weighted')
    #
    print(df_confusion_matrix)
    print(f1_score)

In [ ]:
create_environment()
preprocess_data()
print_object_status(train_labels)
print_object_status(X_train_imputed_scaled)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Environment Done!


NameError: ignored

In [2]:
import os
os.getcwd()
from google.colab import drive 
drive.mount('/content/gdrive')
#chuyển về thư mục data bên trong Google Drive.
os.chdir('/content/gdrive/My Drive/Colab Notebooks/data')
#trong thư mục data này, ta để sẵn các file dữ liệu mà google colab sẽ gọi vào.
os.listdir()
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
#
print('Environment Done!')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Environment Done!


In [ ]:
# import data 
app_train = pd.read_csv('application_train.csv')

#
list_features_top3 = ['TARGET','EXT_SOURCE_2','EXT_SOURCE_1']
app_train = app_train[list_features_top3]

# clean data (encode, impute, dummy)
## Create a label encoder object
le = LabelEncoder()
le_count = 0
# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])                
            # Keep track of how many columns were label encoded
            le_count += 1
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)

if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()    

imputer = Imputer(strategy = 'median')
scaler = MinMaxScaler(feature_range = (0, 1))
imputer.fit(train)
train_imputed = imputer.transform(train)

scaler.fit(train_imputed)
X_train_imputed_scaled = scaler.transform(train_imputed)

train_labels = app_train['TARGET']
#
# return train_labels, X_train_imputed_scaled

In [ ]:
print_object_status(train_labels)
print_object_status(X_train_imputed_scaled)

(307511,)
<class 'pandas.core.series.Series'>
(307511, 1)
<class 'numpy.ndarray'>


In [ ]:
# list_features_top3 = ['EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1']
log_reg = LogisticRegression(C = 0.0001)
run_flow(train_X=X_train_imputed_scaled, train_y=train_labels, algorithm_choice = log_reg)

[[70787     0]
 [ 6091     0]]
0.9587512274404905


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
run_flow(train_X=X_train_imputed_scaled, train_y=train_labels, algorithm_choice = rf)

[[68312  2475]
 [ 5683   408]]
0.9116752397255505


#try1

https://www.kaggle.com/adamsfei/knn-on-application-train-pca-for-ensembling

In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import data 
app_train = pd.read_csv('application_train.csv')

selected_features = ['AMT_INCOME_TOTAL', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 
                     'TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2','EXT_SOURCE_3']
train_df_ltd = app_train[selected_features]
y = train_df_ltd['TARGET']
del train_df_ltd['TARGET']

#let's add one feature that we know performs very well
train_df_ltd['ann_nr'] = train_df_ltd['AMT_CREDIT']/train_df_ltd['AMT_ANNUITY']

pca = PCA()
standard_scaler = StandardScaler() 
train_scaled = standard_scaler.fit_transform(train_df_ltd.fillna(-1))
pca.fit(train_scaled)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [4]:
pca.explained_variance_

array([3.11483194, 1.10999561, 1.03806276, 0.95480371, 0.90376859,
       0.80456935, 0.06219422, 0.01179984])

In [5]:
transformed = pca.transform(train_scaled)
ans = pd.DataFrame(transformed, columns = ['pca_1','pca_2','pca_3','pca_4','pca_5','pca_6', 'pca_7', 'pca_8'])
ans['target'] = y
print('the application train subset projected into the calculated components has the shape of: {}'.format(transformed.shape))

the application train subset projected into the calculated components has the shape of: (307511, 8)


In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
# plt.plot(np.linspace(10, 150, 8, dtype=int), knn_scores)

In [11]:
# test_scaled = standard_scaler.transform(test_df_ltd.fillna(-1))
# test_pca = pca.transform(test_scaled)

X_train,X_test,y_train,y_test=train_test_split(
                                        ans.drop(['target'], axis=1), #train_imputed_scaled,
                                        y, #train_labels,
                                        test_size=0.25,
                                        random_state=0)

model = KNeighborsRegressor(n_neighbors=150, weights='distance')
model.fit(X_train, y_train) # clf.fit(ans.drop(['target'], axis=1), y)
predictions = model.predict(X_test)

print ("Algorithm setting: ", model)

###
df_confusion_matrix = metrics.confusion_matrix(y_test, predictions) 
print("Confusion Matrix : \n", df_confusion_matrix) 
###
print ("Accuracy score: ", metrics.accuracy_score(predictions,y_test))
print ("Precision score: ", metrics.precision_score(predictions,y_test))
print ("Recall score: ", metrics.recall_score(predictions,y_test))
print ("F1 score: ", metrics.f1_score(predictions,y_test, average = 'weighted'))
print ("ROC AUC score: ", metrics.roc_auc_score(predictions, y_test))

Algorithm setting:  KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=150, p=2,
                    weights='distance')


ValueError: ignored

In [15]:
from sklearn.neighbors import KNeighborsClassifier

X_train,X_test,y_train,y_test=train_test_split(
                                        ans.drop(['target'], axis=1), #train_imputed_scaled,
                                        y, #train_labels,
                                        test_size=0.25,
                                        random_state=0)

# https://www.kaggle.com/adamsfei/knn-on-application-train-pca-for-ensembling
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
model = KNeighborsClassifier(n_neighbors=150, weights='distance') 
model.fit(X_train, y_train) # clf.fit(ans.drop(['target'], axis=1), y)
predictions = model.predict(X_test)

print ("Algorithm setting: ", model)

###
df_confusion_matrix = metrics.confusion_matrix(y_test, predictions) 
print("Confusion Matrix : \n", df_confusion_matrix) 
###
print ("Accuracy score: ", metrics.accuracy_score(predictions,y_test))
print ("Precision score: ", metrics.precision_score(predictions,y_test))
print ("Recall score: ", metrics.recall_score(predictions,y_test))
print ("F1 score: ", metrics.f1_score(predictions,y_test, average = 'weighted'))
print ("ROC AUC score: ", metrics.roc_auc_score(predictions, y_test))

Algorithm setting:  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=150, p=2,
                     weights='distance')
Confusion Matrix : 
 [[70784     3]
 [ 6088     3]]
Accuracy score:  0.9207705715549311
Precision score:  0.0004925299622393696
Recall score:  0.5
F1 score:  0.958674801821953
ROC AUC score:  0.710401706733271
